### UL_KB -- Linguistic resources

The current UL_KB includes the following knowledge bases


<img src="images/UL_KB_Linguistic.png" alt="Drawing" style="width: 800px;"/>

In [1]:
!pip install SPARQLWrapper

In [43]:
from SPARQLWrapper import SPARQLWrapper, JSON

query_prefix =  ("prefix rrp: <http://www.ibm.com/RRP#> \n"  
                "prefix glo: <http://www.ibm.com/GLO_V2#> \n"
                "prefix ulvn: <http://www.ibm.com/UL_VN#> \n"  
                "prefix ulwn: <http://www.ibm.com/UL_WN#> \n"  
                "prefix ulpb: <http://www.ibm.com/UL_PB#> \n"  
                "prefix ulkb: <http://www.ibm.com/UL_KB#> \n")

#THIS IS THE SERVER 
sparql = SPARQLWrapper("http://goedel.sl.cloud9.ibm.com:9999/blazegraph/namespace/UL_KB_V1")
#THIS IS A LOCAL VERSION (ROSARIO'S)
#sparql = SPARQLWrapper("http://127.0.0.1:9999/blazegraph/namespace/UL_KB_V0")

## Querying the graph in one step: From 'enter.01' in Propbank to Verbnet class and semantic roles
The following query does this in one shot. If a verbnet class exists, it will produce a mapping. Notice that here two classes are mapped. Only the first one has explicit semantic role mappings. 


In [80]:
query_text = """SELECT DISTINCT ?verb ?pbSemRole ?vnVerbLabel ?vnParamLabel WHERE {   
  ?verb rdfs:label "enter.01" . 
  #?verb rrp:inKB rrp:PropBank .
  ?verb rrp:hasParameter ?pbParam . 
  ?pbParam rdfs:label ?pbSemRole . 
  OPTIONAL {
  	?vnVerb rrp:inKB rrp:VerbNet . 
  	?vnVerb rdfs:label ?vnVerbLabel . 
  	?vnVerb rrp:hasComponent ?vnFrame . 
  	?vnFrame rrp:hasComponent ?semPred . 
  	?semPred rrp:hasParameter ?vnParam . 
  	?pbParam rrp:mapsTo ?vnParam . 
    ?vnParam rdfs:label ?vnParamLabel . 
   } 
  } """
        
sparql.setQuery(query_prefix + query_text) 
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:       
       pbSemRole = result["pbSemRole"]["value"] 
       vnVerbLabel = vnParamLabel = ""
       if "vnVerbLabel" in result : 
           vnVerbLabel = result["vnVerbLabel"]["value"]
       if 'vnParamLabel' in result: 
           vnParamLabel = result["vnParamLabel"]["value"]
       print(pbSemRole + "\t(" + vnVerbLabel + ", " + vnParamLabel + ")")

ARG0	(escape-51.1-2, ThemRole(Theme))
ARG1	(escape-51.1-2, ThemRole(Destination))


### Query the KG by string search
If you want to make a query by string, this query may give you more results, but it can help in browsing the KG

In [81]:
query_text = """SELECT DISTINCT  ?label ?pbVerb ?pbSemRole ?vnVerbLabel ?vnSemRole WHERE {
   
  ?pbVerb rdfs:label ?label. 
  FILTER regex(?label, "^enter","i")  . 
  ?pbVerb rrp:hasParameter ?pbParam . 
  ?pbVerb rrp:inKB rrp:PropBank .
  ?pbParam rdfs:label ?pbSemRole .
  
  ?pbVerb rrp:hasMapping ?mapping . 
  ?vnVerb rrp:hasMapping ?mapping .
  
  ?vnVerb rrp:inKB rrp:VerbNet . 
  ?vnVerb rdfs:label ?vnVerbLabel . 
     
  OPTIONAL {  
  ?vnVerb rrp:hasComponent ?vnFrame . 
  ?vnFrame rrp:hasComponent ?vnPred . 
  ?vnPred rrp:hasParameter ?vnParam . 
  ?vnParam rrp:mapsTo ?pbParam . 
  ?vnParam rrp:varName ?vnSemRole 
   } 
  } ORDER BY ?label """
        
sparql.setQuery(query_prefix + query_text) 
sparql.setReturnFormat(JSON)
results = sparql.query().convert()


for result in results["results"]["bindings"]:
       label = result["label"]["value"]
       pbSemRole = result["pbSemRole"]["value"]       
       vnVerbLabel = result["vnVerbLabel"]["value"]       
       if 'vnSemRole' in result: 
           vnSemRole = result["vnSemRole"]["value"]
       else:  
          vnSemRole = ""
       print(label + " " + pbSemRole + "\t(" + vnVerbLabel + ", " + vnSemRole + ")")

enter.01 ARG0	(escape-51.1-2, Theme)
enter.01 ARG1	(escape-51.1-2, Destination)
enter.01 ARG0	(escape-51.1, )
enter.01 ARG1	(escape-51.1, )
enter.01 ARG0	(escape-51.1-1, )
enter.01 ARG1	(escape-51.1-1, )
enter.01 ARG0	(escape-51.1-1-1, )
enter.01 ARG1	(escape-51.1-1-1, )
enter.01 ARG0	(escape-51.1-1-2, )
enter.01 ARG1	(escape-51.1-1-2, )
enter.01 ARG0	(escape-51.1-1-3, )
enter.01 ARG1	(escape-51.1-1-3, )
enter.01 ARG0	(escape-51.1-3, )
enter.01 ARG1	(escape-51.1-3, )
entertain.01 ARG0	(amuse-31.1, Stimulus)
entertain.01 ARG1	(amuse-31.1, Experiencer)
entertain.01 ARG2	(amuse-31.1, )


### Querying the KG in one step-- Verbnet Semantic Predicates (TEXT ONLY)  
Lets look at the semantic predicates in escape-51_1. This query returns the text version of the predicates. These can be parsed by value. 

Notice that the system returns TWO frames that represent two syntactic forms of the verb class 'escape'. In one, neither the Initial_Location, the Trajectory or Destination are known. In the second, the Initial_Location is known ('from France'). The question mark in the predicate variable indicates that it is not present in the frame. Also, the umlaut on top of an event indicates that it is a process that requires a non-zero amount of time. 

The semantic predicate node ID is also included so it can be queried directly, if needed. We output the identifer part of the URI for simplicity's sake. 


In [79]:
query_text = """SELECT DISTINCT  ?example ?semanticPredicate ?predicateText  WHERE {
                  ulvn:escape-51_1_2 rrp:hasComponent ?frame . 
                  ?frame rrp:example ?example . 
                  ?frame rrp:hasComponent ?semanticPredicate . 
                  ?semanticPredicate a rrp:SemanticPredicate . 
                  ?semanticPredicate rrp:textInfo ?predicateText .  
                } ORDER BY ?frame
                  """
        
sparql.setQuery(query_prefix + query_text) 
sparql.setReturnFormat(JSON)
results = sparql.query().convert()


for result in results["results"]["bindings"]:
       predicateText = result["predicateText"]["value"] 
       example = result["example"]["value"]
       semantic_predicate = result["semanticPredicate"]["value"]
       print(example + "\t" + semantic_predicate.split('#')[1] + "\t" + predicateText)

### Step-by-step: Verbnet classes for a propbank verb

The queries that follow explore the mappings between the Propbank verb and the Verbnet verbs and illustrate how the graph is built. 
To query for Verbnet classes associated to "enter.01", the query below can be issued. Here we return the verb names from Verbnet (escape-51.1-1) and the graph node URIs for those verbs (http://www.ibm.com/XXXXX) You can use these identifiers to query the nodes directly. 



In [72]:
 query_text = """SELECT DISTINCT  ?vnClassNODE ?vnClassLabel  WHERE {
                  ?entity rdfs:label "enter.01" . 
                  ?entity rrp:hasMapping ?mapping . 
                  ?vnClassNODE rrp:hasMapping ?mapping . 
                  ?vnClassNODE rrp:inKB rrp:VerbNet . 
                  ?vnClassNODE rdfs:label ?vnClassLabel . 
                } """
        
sparql.setQuery(query_prefix + query_text) 
sparql.setReturnFormat(JSON)
results = sparql.query().convert()


for result in results["results"]["bindings"]:
       vnClassNODE = result["vnClassNODE"]["value"] 
       vnClassLabel = result["vnClassLabel"]["value"] 
       print("Verbnet Class = " + vnClassLabel + " , graph node = " + vnClassNODE)
       

Verbnet Class = escape-51.1 , graph node = http://www.ibm.com/UL_VN#escape-51_1
Verbnet Class = escape-51.1-1 , graph node = http://www.ibm.com/UL_VN#escape-51_1-1
Verbnet Class = escape-51.1-1-1 , graph node = http://www.ibm.com/UL_VN#escape-51_1-1-1
Verbnet Class = escape-51.1-1-2 , graph node = http://www.ibm.com/UL_VN#escape-51_1-1-2
Verbnet Class = escape-51.1-1-3 , graph node = http://www.ibm.com/UL_VN#escape-51_1-1-3
Verbnet Class = escape-51.1-2 , graph node = http://www.ibm.com/UL_VN#escape-51_1-2
Verbnet Class = escape-51.1-3 , graph node = http://www.ibm.com/UL_VN#escape-51_1-3


### Step by step -- Semantic Predicates for a Verbnet class (graph nodes)

In the example above, we have obtained the text representation of the predicates, but these predicates are also stored in their own nodes. The query below allows the user to retrieve these independently. The last number of the predicate node id refers to its order.  

In [88]:
query_text = """SELECT DISTINCT ?example ?operator ?semanticPredicateLabel ?param ?type ?value  WHERE {
                  ulvn:escape_51_1_1 rrp:hasComponent ?frame . 
                  ?frame rrp:example ?example . 
                  ?frame rrp:hasComponent ?semanticPredicate . 
                  ?semanticPredicate a rrp:SemanticPredicate .
                  ?semanticPredicate rdfs:label ?semanticPredicateLabel. 
                  ?semanticPredicate rrp:hasParameter ?param . 
                  OPTIONAL { 
                    ?semanticPredicate rrp:logicOperatorName ?operator .  
                   }
                  ?param rrp:varType ?type . 
                  ?param rrp:varName ?value . 
                  ?semanticPredicate rrp:textInfo ?predicateText .  
                } ORDER BY ?semanticPredicate
                  """
        
sparql.setQuery(query_prefix + query_text) 
sparql.setReturnFormat(JSON)
results = sparql.query().convert()


for result in results["results"]["bindings"]:
       example = result["example"]["value"]
       if not "operator" in result :
            operator = "" 
       else : 
            operator = result["operator"]["value"]
       typeR = result["type"]["value"]
       valueR = result["value"]["value"]
       semantic_predicate_label = result["semanticPredicateLabel"]["value"]
       print(example + "\t" + operator + " " + semantic_predicate_label + "\t" +  " PARAM [" + typeR + ", " + valueR + "])")

The prisoners advanced.	 has_location	 PARAM [Event, e1])
The prisoners advanced.	 has_location	 PARAM [ThemRole, Theme])
The prisoners advanced.	 has_location	 PARAM [ThemRole, Initial_Location])
The prisoners advanced.	 motion	 PARAM [Event, ë2])
The prisoners advanced.	 motion	 PARAM [ThemRole, Theme])
The prisoners advanced.	 motion	 PARAM [ThemRole, Trajectory])
The prisoners advanced.	NOT has_location	 PARAM [Event, e2])
The prisoners advanced.	NOT has_location	 PARAM [ThemRole, Theme])
The prisoners advanced.	NOT has_location	 PARAM [ThemRole, Initial_Location])
The prisoners advanced.	 has_location	 PARAM [Event, e3])
The prisoners advanced.	 has_location	 PARAM [ThemRole, Theme])
The prisoners advanced.	 has_location	 PARAM [ThemRole, Destination])
He came from France.	 has_location	 PARAM [Event, e1])
He came from France.	 has_location	 PARAM [ThemRole, Theme])
He came from France.	 has_location	 PARAM [ThemRole, Initial_Location])
He came from France.	 motion	 PARAM [Event, 

### Use a JSON-like format for the query output

Same query as before but now data is in dict format


In [89]:
 query_text = """SELECT DISTINCT ?example ?operator ?semanticPredicate ?semanticPredicateLabel ?param ?type ?value  WHERE {{
                  ulvn:escape_51_1_1 rrp:hasComponent ?frame . 
                  ?frame rrp:example ?example . 
                  ?frame rrp:hasComponent ?semanticPredicate . 
                  ?semanticPredicate a rrp:SemanticPredicate .
                  ?semanticPredicate rdfs:label ?semanticPredicateLabel. 
                  ?semanticPredicate rrp:hasParameter ?param . 
                  OPTIONAL {{
                    ?semanticPredicate rrp:logicOperatorName ?operator .  
                   }}
                  ?param rrp:varType ?type . 
                  ?param rrp:varName ?value . 
                  ?semanticPredicate rrp:textInfo ?predicateText .  
                }} ORDER BY ?semanticPredicate
                  """

 sparql.setQuery(query_prefix + query_text) 
 sparql.setReturnFormat(JSON)
 results = sparql.query().convert()

 output = []
 thisFrame = {}
 output.append(thisFrame)
 curPredicateID = ""
 thisPredicate = {}
 #PROCESS OUTPUT IN A JSON-FRIENDLY FORMAT
 for result in results["results"]["bindings"]:
       example = result["example"]["value"]
       
       if not 'example' in thisFrame : 
            thisFrame['example'] = example
       if example != thisFrame['example'] : 
          thisFrame = {}         
          output.append(thisFrame)
          thisFrame['example'] = example 
          curPredicateID = ""
       thisPredicateID = result["semanticPredicate"]["value"]
      
       if 'predicates' not in thisFrame: 
          thisFrame['predicates'] = []        
       predicates = thisFrame['predicates']
    
       if thisPredicateID != curPredicateID : 
           thisPredicate = {}
           predicates.append(thisPredicate)
           curPredicateID = thisPredicateID
           
       #predicateText = result["predicateText"]["value"]
       predLabel = result["semanticPredicateLabel"]["value"]
       
       thisPredicate['label'] = predLabel
       if "operator" in result :      
           thisPredicate['operator'] = result["operator"]["value"] 
           
       if 'params' not in thisPredicate: 
           thisPredicate['params'] = []
       params = thisPredicate['params']
       params.append({'type' :result["type"]["value"], 'value' : result["value"]["value"] , 'predicate' : curPredicateID})
       #thisPredicate['type'] = result["type"]["value"]
       #thisPredicate['value'] = result["value"]["value"]         
       #print(example + " " + " " +  thisPredicateID + " " +  predLabel + " " +  
       #      result["type"]["value"] + " " + result["value"]["value"] )
 print(str(output))               

[{'example': 'The prisoners advanced.', 'predicates': [{'label': 'has_location', 'params': [{'type': 'Event', 'value': 'e1', 'predicate': 'http://www.ibm.com/UL_VN#escape_51_1_1_0_1_fr_1155_pred_1'}, {'type': 'ThemRole', 'value': 'Theme', 'predicate': 'http://www.ibm.com/UL_VN#escape_51_1_1_0_1_fr_1155_pred_1'}, {'type': 'ThemRole', 'value': 'Initial_Location', 'predicate': 'http://www.ibm.com/UL_VN#escape_51_1_1_0_1_fr_1155_pred_1'}]}, {'label': 'motion', 'params': [{'type': 'Event', 'value': 'ë2', 'predicate': 'http://www.ibm.com/UL_VN#escape_51_1_1_0_1_fr_1155_pred_2'}, {'type': 'ThemRole', 'value': 'Theme', 'predicate': 'http://www.ibm.com/UL_VN#escape_51_1_1_0_1_fr_1155_pred_2'}, {'type': 'ThemRole', 'value': 'Trajectory', 'predicate': 'http://www.ibm.com/UL_VN#escape_51_1_1_0_1_fr_1155_pred_2'}]}, {'label': 'has_location', 'operator': 'NOT', 'params': [{'type': 'Event', 'value': 'e2', 'predicate': 'http://www.ibm.com/UL_VN#escape_51_1_1_0_1_fr_1155_pred_3'}, {'type': 'ThemRole', 

### Step by step ---- Mapping Propbank predicates to Verbnet Predicates

In order to obtain the mappings for the semantic roles of a Propbank verb we use the query below. 

In [105]:
query_text = """SELECT DISTINCT  ?otherEntityLabel ?provenance ?KB  WHERE {
                  ?entity rdfs:label "enter.01" . 
                  ?entity rrp:hasMapping ?mapping . 
                  ?otherEntity rrp:hasMapping ?mapping .
                  ?otherEntity rrp:provenance ?provenance . 
                  ?otherEntity rrp:inKB ?KB .               
                  ?otherEntity rdfs:label ?otherEntityLabel. 
                } ORDER BY ?otherEntity
                  """
        
sparql.setQuery(query_prefix + query_text) 
sparql.setReturnFormat(JSON)
results = sparql.query().convert()


for result in results["results"]["bindings"]:
       sense = result["otherEntityLabel"]["value"] 
       provenance = result["provenance"]["value"]
       KB = result["KB"]["value"]
       print(sense + "\t" + provenance + "\t\t" + KB.split("#")[1])

advance.01	Propbank NLTK		PropBank
approach.01	Propbank NLTK		PropBank
arrive.01	Propbank NLTK		PropBank
ascend.01	Propbank NLTK		PropBank
bolt.01	Propbank NLTK		PropBank
climb.01	Propbank NLTK		PropBank
come.01	Propbank NLTK		PropBank
cross.02	Propbank NLTK		PropBank
depart.01	Propbank NLTK		PropBank
descend.01	Propbank NLTK		PropBank
disembark.01	Propbank NLTK		PropBank
emerge.01	Propbank NLTK		PropBank
enter.01	Propbank NLTK		PropBank
escape.01	Propbank NLTK		PropBank
exit.01	Propbank NLTK		PropBank
fall.01	Propbank NLTK		PropBank
file.03	Propbank NLTK		PropBank
go.01	Propbank NLTK		PropBank
go.02	Propbank NLTK		PropBank
plunge.02	Propbank NLTK		PropBank
recede.01	Propbank NLTK		PropBank
retreat.01	Propbank NLTK		PropBank
return.01	Propbank NLTK		PropBank
rise.01	Propbank NLTK		PropBank
tumble.01	Propbank NLTK		PropBank
vacate.01	Propbank NLTK		PropBank
withdraw.01	Propbank NLTK		PropBank
escape-51.1-1	verbnet3.4		VerbNet
escape-51.1-1-1	verbnet3.4		VerbNet
escape-51.1-1-2	verbnet3.

In [40]:
className = "absorb-39.8"

In [42]:
print(className.split('-', 1)[1])

39.8
